# PUI Homework 4
## Part I
### By Max Feinglass

In [ ]:
import requests
import pandas as pd
import zipfile
import StringIO
import scipy.stats as sci
import numpy as np

np.random.seed(1984)


Use: age of bikers for male and age of bikers for female gender for whichever month, or set of months you choose. 
test the correlation/independence of the 2 samples using: 
Pearson’s test (answer: are the 2 samples correlated?)
Spearman’s test (answer: are the 2 samples correlated?)
K-S test (answer: are the samples likely to come from the same parent distribution?) 
State in words what the tests tell you.

Use: age of bikers during the day and during the night hours for the same sample .
Test the correlation/independence of the 2 samples using the same tests.
State in words what the tests tell you.

### Load Data Frame from Web for July, 2015

In [ ]:
#Load September 2014 data in Pandas dataframe import requests, zipfile, StringIO 
r = requests.get('https://s3.amazonaws.com/tripdata/201507-citibike-tripdata.zip') 
z = zipfile.ZipFile(StringIO.StringIO(r.content)) 
df = pd.read_csv(z.open('201507-citibike-tripdata.csv'), parse_dates=['starttime'])


## Throw Out Unreliable Data

In [ ]:
# Get Range for Trip Duration Outliers
tripstd = df.tripduration.std()
tripmean = df.tripduration.mean()
# Low limit is 2 minutes
llimit = 120
# High limit is 2 STD from mean
hlimit = tripmean + (2 * tripstd)

In [ ]:
# Throw out trips of less than 2 minutes and very long trips (>2 STD from mean)
df['ageM'] = df['birth year'][(df['tripduration']>=llimit) & (df['tripduration']<=hlimit)]
df['ageW'] = df['birth year'][(df['tripduration']>=llimit) & (df['tripduration']<=hlimit)]

df['night'] = df['birth year'][(df['tripduration']>=llimit) & (df['tripduration']<=hlimit)]
df['day'] = df['birth year'][(df['tripduration']>=llimit) & (df['tripduration']<=hlimit)]


In [ ]:
# Throw out ages that are greater than 70 and lower than 10
df['ageM'] = df['ageM'][(df['birth year']<= 2005) & (df['birth year']>= 1935)]
df['ageW'] = df['ageM'][(df['birth year']<= 2005) & (df['birth year']>= 1935)]

df['night'] = df['night'][(df['birth year']<= 2005) & (df['birth year']>= 1935)]
df['day'] = df['day'][(df['birth year']<= 2005) & (df['birth year']>= 1935)]


In [ ]:
# Create age of gender columns
df['ageM'] = 2015-df['birth year'][(df['usertype'] == 'Subscriber') & (df['gender'] == 1)]
df['ageW'] = 2015-df['birth year'][(df['usertype'] == 'Subscriber') & (df['gender'] == 2)]

df['night'] = 2015-df['birth year'][(df['usertype'] == 'Subscriber') & ((df.starttime.dt.hour <= 5) | (df.starttime.dt.hour >= 8+12))] 
df['day'] = 2015-df['birth year'][(df['usertype'] == 'Subscriber') & ((df.starttime.dt.hour >= 5) & (df.starttime.dt.hour <= 8+12))]

### Drop out NaN values and make array's the same size

In [ ]:
men = df.ageM.dropna()
women = df.ageW.dropna()

night = df.night.dropna()
day = df.day.dropna()

menselect = np.random.choice(men, len(women), 'false')
womenselect = np.random.choice(men, len(women), 'false')

nightselect = np.random.choice(night, len(night), 'false')
dayselect = np.random.choice(day, len(night), 'false')

print nightselect
print dayselect
print len(nightselect), len(dayselect)


### Pearson's Correlation Test of the ages of male and female bikers

In [ ]:
sci.stats.pearsonr(menselect, womenselect)

#### The person coefficient test is a conjunction of a “simple” correlation between two variables and then a test of the probability that this correlation arose from stochastic processes.  Here, the correlation is .001, which is very close to zero.  This is expected as there is no obvious reason the age of men is connected to the age of women.  The P-value, of .406 percent, is a sense that it is very possible that the result arose from stochastic chance and therefore should be taken with ‘a grain of salt’.

### Spearman's Test

In [ ]:
sci.stats.spearmanr(menselect, womenselect)

#### The Spearman test is an examination of the tendency of a value in one array to have the same sign as another variable in another array.  This tests the monotonicity of the probability density function of a distribution.  Unlike Pearson, the Spearman test checks if the relationship between the two PDFs are increasing or decreasing, even if that increase or decrease is nonlinear.  Here, there is no correlation between the variables and the chances that this result arose stochastically is relatively high.

### K-S Test

In [ ]:
sci.stats.ks_2samp(menselect, womenselect)

#### The KS (Kolmogorov–Smirnov) Test compares the cumulative distribution functions of two distributions.  It is a measure of the distance between the two and can be used to asses if the two dstirubtions are of the same type.  If the K-S statistic is small or the p-value is high, then we cannot reject the hypothesis that the distributions of the two samples are the same.  Here the KS statistic is quite small and the probability is very high that the statistic arose stochastically.  Therefore we cannot say the observations are the same distribution.  

### Pearson's Correlation Test of the ages of day and night time bikers

In [ ]:
sci.stats.pearsonr(dayselect, nightselect)

#### The person coefficient test is a conjunction of a “simple” correlation between two variables and then a test of the probability that this correlation arose from stochastic processes.  Here, the correlation is .005,  is very close to zero.  The P-value, of 0.04 percent, implies you CAN make claim that day and night values are NOT correlated.  

### Spearman's Test

In [ ]:
sci.stats.spearmanr(dayselect, nightselect)

#### The Spearman test is an examination of the tendency of a value in one array to have the same sign as another variable in another array. This tests the monotonicity of the probability density function of a distribution. Unlike Pearson, the Spearman test checks if the relationship between the two PDFs are increasing or decreasing, even if that increase or decrease is nonlinear. Here, there is no correlation between the variables and the chances that this result arose stochastically is low, but still possible 10% of the time.  It the values are very close to the person values.  

### K-S Test

In [ ]:
sci.stats.ks_2samp(dayselect, nightselect)

#### If the K-S statistic is small or the p-value is high, then we cannot reject the hypothesis that the distributions of the two samples are the same.  Here the KS statistic is quite small and the probability is zero that the statistic arose stochastically.  So even though the P-value is low (these are empirical samples so they could never arrive stochastically) the low KS score implies one cannot reject the null hypothesis.